In [2]:
import pandas as pd
path = r'C:\Users\Lenovo\Desktop\Tool programming\homework\Tourism_YTDChange_Completed.xlsx'
final_df_allmonths = pd.read_excel(path)

In [3]:
final_df_allmonths.head()

,Region,Country,Year,Month,Value,%Change_YTD
0,South-East Asia,Brunei,2025,Jan,1306,-6.61
1,South-East Asia,Cambodia,2025,Jan,46001,-6.61
2,South-East Asia,Indonesia,2025,Jan,82919,3.91
3,South-East Asia,Laos,2025,Jan,94271,-14.08
4,South-East Asia,Malaysia,2025,Jan,443015,-1.29


In [ ]:
import plotly.express as px

# ฟิลเตอร์เฉพาะข้อมูลปี 2025 และข้อมูลที่มี %Change_YTD
ytd_data = final_df_allmonths[(final_df_allmonths['Year'] == 2025) & (final_df_allmonths['%Change_YTD'].notna())]

# ลบค่าซ้ำให้เหลือเพียงค่า %Change_YTD ต่อประเทศเดียว
ytd_unique = ytd_data.drop_duplicates(subset=['Country'])

# 1. กราฟเปรียบเทียบ %Change_YTD ต่อภูมิภาค
fig_region = px.box(ytd_unique, x='Region', y='%Change_YTD',
                    title='%Change_YTD Distribution by Region (2025)',
                    points="all", color='Region')

# 2. Top ประเทศที่เติบโตสูงสุดในไตรมาสแรก
top_countries = ytd_unique.sort_values(by='%Change_YTD', ascending=False).head(10)
fig_top10 = px.bar(top_countries, x='Country', y='%Change_YTD',
                   color='Region', text='%Change_YTD',
                   title='Top 10 Countries by %Change_YTD (2025)',width=1000, height=600)

fig_top10.update_traces(texttemplate='%{text:.2f}%', textposition='outside')

# 3. Heatmap ของ %Change_YTD ทั่วโลก
fig_heatmap = px.choropleth(ytd_unique, locations='Country', locationmode='country names',
                            color='%Change_YTD', hover_name='Country',
                            color_continuous_scale='RdYlGn',
                            title='%Change_YTD by Country (2025)',width=1000, height=700)

fig_region.show()
fig_top10.show()
fig_heatmap.show()


In [ ]:
# Load the newly uploaded file
df = pd.read_excel(r'C:\Users\Lenovo\Desktop\Tool programming\homework\Tourism_YTDChange_Completed.xlsx')

# Filter for year 2025
df_2025 = df[df['Year'] == 2025]


# Monthly Value comparison for South-East Asia
sea_monthly = df_2025[df_2025['Region'] == 'South-East Asia']

# ASEAN countries line chart over Jan-Mar 2025
asean_countries = [
    "Brunei", "Cambodia", "Indonesia", "Laos", "Malaysia",
    "Myanmar", "Philippines", "Singapore", "Thailand", "Vietnam"
]
asean_df = df_2025[df_2025['Country'].isin(asean_countries)]


# Monthly Value comparison for South-East Asia
fig_sea = px.bar(sea_monthly, x='Month', y='Value',
                 color='Country', barmode='group',
                 title='Monthly Tourist Volume by Country (South-East Asia, Jan-Mar 2025)')

# ASEAN Line Chart
fig_asean = px.line(asean_df, x='Month', y='Value', color='Country',
                    markers=True, title='ASEAN Country Trends Jan-Mar 2025')


fig_sea.show()
fig_asean.show()


In [ ]:
mar_2025 = df_2025[df_2025['Month'] == 'Mar']
top10_mar = mar_2025.sort_values(by='Value', ascending=False).head(10)
top10_mar_cleaned = mar_2025[~mar_2025['Country'].str.contains("Total", case=False)]
# 1. Prepare data for Pareto Chart (Top 10 by Value in Mar 2025)
pareto_df = top10_mar_cleaned.sort_values(by='Value', ascending=False).copy()
pareto_df['Cumulative'] = pareto_df['Value'].cumsum()
pareto_df['Cumulative %'] = 100 * pareto_df['Cumulative'] / pareto_df['Value'].sum()


# 3. Bubble Chart (Mar 2025: Value vs %Change_YTD)
mar_bubble = mar_2025[mar_2025['%Change_YTD'].notna()]

# Bubble Chart: Value vs %Change_YTD
fig_bubble = px.scatter(mar_bubble, x='%Change_YTD', y='Value', size='Value',
                        color='Region', hover_name='Country',
                        title='Bubble Chart: %Change_YTD vs Value (Mar 2025)',
                        size_max=60)
fig_bubble.show()




In [23]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# 1. Top 5 Countries with Momentum Drop in March (Value Mar < Jan-Feb Avg)
growth_df = df_2025[df_2025['Month'].isin(['Jan', 'Feb', 'Mar'])]
growth_summary = growth_df.groupby(['Country', 'Month'])['Value'].sum().unstack()
growth_summary['Jan-Feb Avg'] = growth_summary[['Jan', 'Feb']].mean(axis=1)
growth_summary['Mar vs Jan-Feb Drop %'] = ((growth_summary['Mar'] - growth_summary['Jan-Feb Avg']) / growth_summary['Jan-Feb Avg']) * 100
momentum_drop = growth_summary.sort_values(by='Mar vs Jan-Feb Drop %').dropna().head(5)

# 2. Total Value by Region (Jan-Mar 2025)
region_summary = df_2025.groupby(['Region', 'Month'])['Value'].sum().unstack()
region_summary['Total Jan-Mar'] = region_summary.sum(axis=1)



In [28]:
# 1. Plot Momentum Drop Countries as Bar Chart
fig_drop = px.bar(momentum_drop.reset_index(), x='Country', y='Mar vs Jan-Feb Drop %',
                  title='Top 5 Countries with Momentum Drop in March 2025',
                  text='Mar vs Jan-Feb Drop %',color='Country',width=1100, height=600)
fig_drop.update_traces(texttemplate='%{text:.2f}%', textposition='outside')

# 2. Region Total Comparison - Grouped Bar by Month
region_summary_reset = region_summary.reset_index().melt(id_vars='Region', var_name='Month', value_name='Value')
fig_region = px.bar(region_summary_reset[region_summary_reset['Month'] != 'Total Jan-Mar'],
                    x='Region', y='Value', color='Month', barmode='group',
                    title='Monthly Tourist Volume by Region (Jan-Mar 2025)')

fig_region.update_layout(xaxis_tickangle=-45)
fig_drop.show()
fig_region.show()

